<a href="https://colab.research.google.com/github/NetoRibeiro/DATA8001Assignment1/blob/main/Linear_Regression_Model_11042021_002_ETL_Int.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import and Load File

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import pandas as pd
from datetime import datetime as dt

import calendar
import re
import string

import math
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

from scipy import stats

In [2]:
df_processed = pd.read_csv('/content/drive/MyDrive/Data Science and Analytics/R00206995/data/R00206995_processed.csv')

In [3]:
df_processed.head()

,car_reg,purchase_date,year,month,county,make,model,type,colour,tax_band,price
0,202-C-2315,2020-07-01,2020,7,CORK,AUDI,A4,SALOON,RED,B,55287.0
1,191-C-3750,2019-01-20,2019,1,CORK,MAZDA,CX-30,SUV,SILVER,B,41690.0
2,191-L-3155,2019-03-21,2019,3,LIMERICK,BMW,3 SERIES,SALOON,WHITE,C,40381.0
3,191-D-2645,2019-01-26,2019,1,DUBLIN,AUDI,Q3,SUV,ORANGE,C,44836.0
4,181-W-341,2018-01-27,2018,1,WATERFORD,OPEL,ASTRA,HATCHBACK,RED,A,32188.0


###Transformation and Split

In [4]:
def get_feature_stats(df, list_columns):
    lis_of_string_features = []
    for feature in list_columns:
      if df[feature].dtype not in ['uint8', 'int16','int32','int64','float16','float32','float64']:
        lis_of_string_features.append(feature)
    return lis_of_string_features


def concat_columns(col1, col2, delimited):
  try:
    cols = f'{col1}{delimited}{col2}'
    return cols
  except Exception as ex:
    raise Exception(f'Error when concatenate:: {ex}')


def apply_calc_columns(col1, col2, calc):
  try:
    col1 = float(col1)
    col2 = float(col2)
 
    if calc == 'times':
      cols = col1 * col2
 
    elif calc == 'div':
      try:
        cols == col1 / col2
      except Exception as ex:
        raise Exception(f'Error when try to divide:: {ex}')
        return 0.0
 
    elif calc == 'plus':
      cols = col1 + col2
 
    elif calc == 'minus':
      cols = col1 - col2
 
    else:
      cols = 0

    return cols
  except Exception as ex:
    raise Exception(f'Error when convert to a float:: {ex}')


In [5]:
df_concats = df_processed.copy()

In [6]:
df_concats.drop(['car_reg', 'purchase_date', 'month'], axis=1, inplace=True)

In [7]:
list_columns = df_concats.columns
list_feature = get_feature_stats(df_concats, list_columns)

In [8]:
df_train, df_test = train_test_split(df_concats.copy() ,test_size=0.2, random_state=8001)

In [9]:
df_concats.head(3)

,year,county,make,model,type,colour,tax_band,price
0,2020,CORK,AUDI,A4,SALOON,RED,B,55287.0
1,2019,CORK,MAZDA,CX-30,SUV,SILVER,B,41690.0
2,2019,LIMERICK,BMW,3 SERIES,SALOON,WHITE,C,40381.0


In [10]:
df_train.head(3)

,year,county,make,model,type,colour,tax_band,price
654,2018,DUBLIN,MAZDA,CX-30,SUV,WHITE,C,41920.0
1947,2018,WATERFORD,FORD,KUGA,SUV,WHITE,B,34961.0
3698,2020,DUBLIN,BMW,3 SERIES,SALOON,BLUE,B,69784.0


In [11]:
list_feature

['county', 'make', 'model', 'type', 'colour', 'tax_band']

In [13]:
#Convert all string features to a int features
for feature in list_feature:
  label_condition = LabelEncoder()
  df_train[feature] = 1 + label_condition.fit_transform(df_train[feature])
  df_test[feature] = 1 + label_condition.fit_transform(df_test[feature])
  df_concats[feature] = 1 + label_condition.fit_transform(df_concats[feature])

In [14]:
#df_train.head(3)
df_concats.head(3)

,year,county,make,model,type,colour,tax_band,price
0,2020,1,1,6,4,3,2,55287.0
1,2019,1,4,15,5,4,2,41690.0
2,2019,4,2,2,4,5,3,40381.0


In [15]:
label_condition.classes_ 

array(['A', 'B', 'C', 'D', 'E'], dtype=object)

In [17]:
df_concats.to_csv('/content/drive/MyDrive/Data Science and Analytics/R00206995/data/R00206995_basic_tofit.csv', index=False)

In [19]:
# X variables (independent)
output_feature_dummies = 'price'
input_features_new_fit = [item for item in df_concats.columns if item not in output_feature_dummies]
print(input_features_new_fit)

['year', 'county', 'make', 'model', 'type', 'colour', 'tax_band']


In [20]:
# X variables (independent)
output_feature_dummies = 'price'
input_features_dummies = [item for item in df_train.columns if item not in output_feature_dummies]
print(input_features_dummies)

['year', 'county', 'make', 'model', 'type', 'colour', 'tax_band']


In [24]:
# Apply Standar Scaler for a list of features
for column in df_concats.columns:

  if column in input_features_new_fit:
    scaler_column = StandardScaler()
    scaler_column.fit(df_concats[column].values.reshape(-1, 1))
    df_concats[column] = scaler_column.transform(df_concats[column].values.reshape(-1, 1))
      
  else:
    print(f'skip:\t{column}')
df_concats.to_csv('/content/drive/MyDrive/Data Science and Analytics/R00206995/data/R00206995_basic_std_tofit.csv', index=False)

skip:	price


In [25]:
#df_dummies.head(3)

In [26]:
# Apply Standar Scaler for a list of features
for column in df_train.columns:

  if column in input_features_dummies:
    scaler_column = StandardScaler()
    scaler_column.fit(df_train[column].values.reshape(-1, 1))
    df_train[column] = scaler_column.transform(df_train[column].values.reshape(-1, 1))
    df_test[column] = scaler_column.transform(df_test[column].values.reshape(-1, 1))
    
  
  else:
    print(f'skip:\t{column}')
# df_dummies.to_csv('/content/drive/MyDrive/Data Science and Analytics/R00206995/data/R00206995_tofit_dum.csv', index=False)

skip:	price


###Linear Regressor Model
####Standard Scaler plus Best Features Dummies

In [27]:
X_train = df_train[input_features_dummies].values
X_test = df_test[input_features_dummies].values

y_train = df_train[[output_feature_dummies]].values
y_test = df_test[[output_feature_dummies]].values

In [28]:
df_train.head(3)

,year,county,make,model,type,colour,tax_band,price
654,-1.254703,-0.411111,-0.126048,-0.071977,1.443934,0.968045,1.086334,41920.0
1947,-1.254703,2.018788,-0.638329,0.728682,1.443934,0.968045,-0.206212,34961.0
3698,1.195070,-0.411111,-1.150611,-1.558914,0.536813,-1.661267,-0.206212,69784.0


In [29]:
lr_scaler_model = LinearRegression()
lr_scaler_model.fit(X=X_train, y=y_train)
lr_scaler_model_prodict = lr_scaler_model.predict(X=X_test)
lr_scaler_model_mse = mean_squared_error(y_pred=lr_scaler_model_prodict, y_true=y_test)
lr_scaler_model_r_sq = r2_score(y_pred=lr_scaler_model_prodict, y_true=y_test)

###Valitation Test:: Linear Regressor Model Standard Scaler and Best Features Dummies

In [30]:
print(f'Linear Model StandardScaler Features Accuracy:\nRMSE={math.sqrt(lr_scaler_model_mse):,.0f}\nRSq={lr_scaler_model_r_sq:.2f} or {lr_scaler_model_r_sq*100:.0f}% of the variability in Y can be explained using X')

Linear Model StandardScaler Features Accuracy:
RMSE=22,204
RSq=0.45 or 45% of the variability in Y can be explained using X


In [31]:
lr_scaler_model.coef_

array([[ 1024.98690391,   266.46766557, -3547.74220955, -5740.17599986,
         1193.14092475,  -394.59367191, 15307.99966497]])

In [ ]:
lr_scaler_model.score

<bound method RegressorMixin.score of LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)>

In [ ]:
lr_scaler_model.intercept_, lr_scaler_model.rank_

(array([46814.18519737]), 9)

In [ ]:
#pickle.dump(lr_scaler_model, open('/content/drive/MyDrive/Data Science and Analytics/R00206995/model/model.pkl', 'wb') )

In [ ]:
shapiro_test = stats.shapiro(X_train)
shapiro_test

/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1676: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


(0.873183012008667, 0.0)